### Funkcja `super()`

Gdy podklasa nadpisuje metodę klasy podrzędnej, metoda zastępująca zazwyczaj potrzebuje sposobu wywołania odpowiadającej jej metody klasy nadrzędnej. Zalecany sposób realizacji tego zadania jest następujący.

In [1]:
import collections

class LastUpdatedOrderedDict(collections.OrderedDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, value)
        self.move_to_end(key)

Klasa `LastUpdatedOrderedDict` tak nadpisuje metodę `__setitem__` aby używała `super().__setitem__` do wywoływania tej metody w klasie nadrzędnej, aby umożliwić wstawianie albo aktualizację pary klucz/wartość.

Funkcja `super()` implementuje logikę pozwalającą obsłużyć hierarchie klas z wielokrotnym dziedziczeniem. 

In [2]:
class LastUpdatedOrderedDict(collections.OrderedDict):
    def __setitem__(self, key, value):
        """To działa w Pythonie 2 i Pythonie 3."""
        super(LastUpdatedOrderedDict, self).__setitem__(key, value)
        self.move_to_end(key)

Obecnie (Python 3) oba argumenty funkcji `super()` są opcjonalne. Kompilator dostarcza je automatycznie po zbadaniu otaczającego kontekstu, gdy funkcja `super()` jest wywoływana w metodzie. Tymi argumentami są:
- typ - początek ścieżki wyszukiwania klasy nadrzędnej implementującej pożądaną metodę. Domyślnie jest to klasa posiadająca metodę, w której występuje wywołanie funkcji `super()`
- obiekt_lub_typ - obiekt (w wywołaniach metod instancji) lub klasa (w wywołaniach metod klasy), która ma być odbiorcą wywołania metody. Domyślnie jest to `self` jeśli wywołanie następuje w metodzie instancji.

Wywołanie funkcji `super()` zwraca dynamiczny obiekt proxy, który odnajduje metodę w klasie nadrzędnej zgodnej z parametrem (typ) i wiąże ją z (obiekt_lub_typ).

### Wielokrotne dziedziczenie i kolejność ustalania metod

In [10]:
class Root:
    def ping(self):
        print(f"{self}.ping() in Root")

    def pong(self):
        print(f"{self}.pong() in Root")

    def __repr__(self):
        return f"<instance of {type(self).__name__}>"


class A(Root):
    def ping(self):
        print(f"{self}.ping() in A")
        super().ping()

    def pong(self):
        print(f"{self}.pong() in A")
        super().pong()


class B(Root):
    def ping(self):
        print(f"{self}.ping() in B")
        super().ping()

    def pong(self):
        print(f"{self}.pong() in B")
        # Nie ma super()!


class Leaf(A, B):
    def ping(self):
        print(f"{self}.ping() in Leaf")
        super().ping()

    # Nie ma pong()!

In [11]:
leaf1 = Leaf()
leaf1.ping()

<instance of Leaf>.ping() in Leaf
<instance of Leaf>.ping() in A
<instance of Leaf>.ping() in B
<instance of Leaf>.ping() in Root


In [12]:
leaf1.pong()

<instance of Leaf>.pong() in A
<instance of Leaf>.pong() in B


Sekwencje aktywowania metod są determinowane przez dwa czynniki:
- kolejność ustalania metod w klasie `Leaf`
- użycie `super()` w każdej metodzie

Kolejność ustalania metod (method resolution order - mro) jest obliczane za pomocą algorytmu C3. Każda klasa ma atrybut o nazwie `__mro__`, który przechowuje krotkę zawierającą referencje do klas nadrzędnych w kolejności ustalania metod, w górę od klasy bieżącej aż do klasy `object`.

In [13]:
Leaf.__mro__

(__main__.Leaf, __main__.A, __main__.B, __main__.Root, object)

MRO ustala jedynie kolejność wywoływania, natomiast to, czy metoda zostanie aktywowana w każdej z tych klas, żależy od tego czy implementacja każdej z nich wywołuje `super()` czy nie. 

Zwróćmy uwagę na metodę `pong()`. Klasa `Leaf` jej nie przesłania, zatem wywołanie `leaf.pong()` aktywuje impementację zawartą w kolejnej klasie wymienionej przez MRO. Jest to klasa `A`. Metoda `A.pong()` wywołuje `super().pong()`. Klasa `B` jest następna w kolejności zatem aktywuje się metoda `B.pong()`. Jednak metoda `B.pong()` nie wywołuje `super().pong()` także sekwencja wywołań kończy się w tym miejscu.

MRO bierze pod uwagę nie tylko graf dziedziczenia ale kolejność w jakiej klasy są wymienione w deklaracji podklasy.

In [14]:
class Leaf(B, A):
    def ping(self):
        print(f"{self}.ping() in Leaf")
        super().ping()

In [15]:
leaf2 = Leaf()
leaf2.ping()

<instance of Leaf>.ping() in Leaf
<instance of Leaf>.ping() in B
<instance of Leaf>.ping() in A
<instance of Leaf>.ping() in Root


In [16]:
leaf2.pong()

<instance of Leaf>.pong() in B


In [17]:
Leaf.__mro__

(__main__.Leaf, __main__.B, __main__.A, __main__.Root, object)

Gdy metoda wywołuje `super()`, nazywamy ją metodą kooperatywną. Takie metody umożliwiają mechanizm kooperatywnego dziedziczenia wielokrotnego. Wielokrotne dziedziczenie wymaga aktywnej współpracy metod. W klasie `B` metoda `ping()` współpracuje ale metoda `pong()` nie. Zalecane jest aby każda metoda w klasie nie będącej korzeniem wywoływała `super()`. Metody kooperatywne muszą mieć kompatybilne sygnatury, gdyż nie wiemy czy `A.ping()` wywoła się przed czy po `B.ping()`.

### Klasy domieszkowe

Klasa domieszkowa (mixin class) jest projektowana w celu tworzenia podklas wspólnie z co najmniej jedną inną klasą w toku dziedziczenia wielokrotnego. Klasa taka dodaje lub dostosowuje pewne zachowania klas potomnych lub spokrewnionych. 

In [19]:
def _upper(key):
    try:
        return key.upper()
    except AttributeError:
        return key


class UpperCaseMixin:
    def __setitem__(self, key, value):
        super().__setitem__(_upper(key), value)  # type: ignore

    def __getitem__(self, key):
        return super().__getitem__(_upper(key))  # type: ignore

    def __contains__(self, key):
        return super().__contains__(_upper(key))  # type: ignore

    def get(self, key, default=None):
        return super().get(_upper(key), default)  # type: ignore

Ponieważ każda metoda w `UpperCaseMixin` wywołuje `super()`, ta klasa domieszkowa zależna jest od klas pokrewnych, które implementują lub dziedziczą metody o takiej samej sygnaturze. Aby uwzględnić jej wkład, musi ona wystąpić w MRO używającej jej klasy podrzędnej przed innymi klasami. Prkatycznie oznacza to, że klasa domieszki musi występować jako pierwsza w krotce klas bazowych w deklaracji klasy.

In [20]:
class UpperDict(UpperCaseMixin, collections.UserDict):
    pass


class UpperCounter(UpperCaseMixin, collections.Counter):
    pass

In [24]:
d = UpperDict({"abc": 1, "2": "two"})
list(d.keys())

['ABC', '2']

In [25]:
d["bbb"] = "letter b"
d

{'ABC': 1, '2': 'two', 'BBB': 'letter b'}

In [29]:
"bbb" in d, d["abc"], d.get("bbb")

(True, 1, 'letter b')

In [30]:
c = UpperCounter("Abra kadabra")
c.most_common()

[('A', 5), ('B', 2), ('R', 2), (' ', 1), ('K', 1), ('D', 1)]

Nie używaj dziedziczenia.